# Eye Blink Detection Framework

Implements a blink detection tracking framework based on facial landmarks

- Uses dlib (2d detection in Grey Color Space) with the shape_predictor_68_face_landmarks model. [1,4]

- Uses mediapipe for debug face mesh output overlay (can be disabled for performance sensitive deployments) [3]


## Logic

- If eyes closed is detected for NOTIFICATION_SECONDS (e.g. 4), a telegram alert is dispatched
- After alert has been dispatched, notifications are blockedun less the user unblocks the system by closing eyes again for NOTIFICATION_SECONDS


## Issues

- Glasses can interfere with detection
- Racial features / skin tones may affect performances.

## Future Improvements


- switch from dlib to mediapipe [2] for detection (works in 3d space, doesn't need beefy model download). 
  - This should provide superior performance, especially for low contrast situations

## Requirements

```
python 3 + pip
python-dotenv
opencv-python (cv2)
mediapipe
dlib
```

## Installation

1. Create a .env file with 

```
BOT_TOKEN=Your telegram bot token
CHAT_ID=Your telegram chat recipient
```
2. Run the notebook

3. Profit

## References

[1] https://medium.com/algoasylum/blink-detection-using-python-737a88893825
[2] https://towardsdatascience.com/face-landmark-detection-using-python-1964cb620837
[3] https://www.analyticsvidhya.com/blog/2021/07/facial-landmark-detection-simplified-with-opencv/
[4] https://www.pyimagesearch.com/2017/04/24/eye-blink-detection-opencv-python-dlib/


In [7]:

NOTIFICATION_SECONDS = 4

import os
from dotenv import load_dotenv
from pathlib import Path  # Python 3.6+ only
env_path = '.env.local'
load_dotenv(dotenv_path=env_path)

bot_token = os.environ['BOT_TOKEN']
bot_chatID = os.environ['CHAT_ID']

import requests
import time
import cv2
import dlib
import math
import mediapipe as mp

def telegram_bot_sendtext(bot_message):
    

    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()


def telegram_bot_sendimage(image):    
    params = {'chat_id': bot_chatID}
    files = {'photo': image}
    resp = requests.post('https://api.telegram.org/bot' + bot_token + '/sendPhoto', params, files=files)
    return resp




#-----Step 1: Use VideoCapture in OpenCV-----

BLINK_RATIO_THRESHOLD = 5.7

#-----Step 5: Getting to know blink ratio

def midpoint(point1 ,point2):
    return (point1.x + point2.x)/2,(point1.y + point2.y)/2

def euclidean_distance(point1 , point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def get_blink_ratio(eye_points, facial_landmarks):
    
    #loading all the required points
    corner_left  = (facial_landmarks.part(eye_points[0]).x, 
                    facial_landmarks.part(eye_points[0]).y)
    corner_right = (facial_landmarks.part(eye_points[3]).x, 
                    facial_landmarks.part(eye_points[3]).y)
    
    center_top    = midpoint(facial_landmarks.part(eye_points[1]), 
                             facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), 
                             facial_landmarks.part(eye_points[4]))

    #calculating distance
    horizontal_length = euclidean_distance(corner_left,corner_right)
    vertical_length = euclidean_distance(center_top,center_bottom)

    ratio = horizontal_length / vertical_length

    return ratio

#livestream from the webcam 
cap = cv2.VideoCapture(0)

'''in case of a video
cap = cv2.VideoCapture("__path_of_the_video__")'''

#name of the display window in OpenCV
cv2.namedWindow('BlinkDetector')

#-----Step 3: Face detection with dlib-----
detector = dlib.get_frontal_face_detector()


mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=1)

#-----Step 4: Detecting Eyes using landmarks in dlib-----
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")
#these landmarks are based on the image above 
left_eye_landmarks  = [36, 37, 38, 39, 40, 41]
right_eye_landmarks = [42, 43, 44, 45, 46, 47]
blink_count = 0
is_blinking = False
start_time = None
block_notification = False

while True:
    #capturing frame
    retval, frameC = cap.read()

    #exit the application if frame not found
    if not retval:
        print("Can't receive frame (stream end?). Exiting ...")
        break 

    #-----Step 2: converting image to grayscale-----
    
    imgRGB = cv2.cvtColor(frameC, cv2.COLOR_BGR2RGB)
    
    results = faceMesh.process(imgRGB)    

                       


    frame = cv2.cvtColor(frameC, cv2.COLOR_BGR2GRAY)
    if results.multi_face_landmarks:
        for faceLms in results.multi_face_landmarks:
            mpDraw.draw_landmarks(frameC, faceLms,mpFaceMesh.FACEMESH_TESSELATION, drawSpec, drawSpec)

    #-----Step 3: Face detection with dlib-----
    #detecting faces in the frame 
    faces,_,_ = detector.run(image = frame, upsample_num_times = 0, 
                       adjust_threshold = 0.0)


    #-----Step 4: Detecting Eyes using landmarks in dlib-----
    for face in faces:
        
        landmarks = predictor(frame, face)


        #-----Step 5: Calculating blink ratio for one eye-----
        left_eye_ratio  = get_blink_ratio(left_eye_landmarks, landmarks)
        right_eye_ratio = get_blink_ratio(right_eye_landmarks, landmarks)
        blink_ratio     = (left_eye_ratio + right_eye_ratio) / 2

        if blink_ratio > BLINK_RATIO_THRESHOLD:
            if not is_blinking: 
                #Blink detected! Do Something!
                blink_count+=1                
                start_time = time.time()
                is_blinking = True
            else:
                elapsed = time.time() - start_time
                if not block_notification:
                    cv2.putText(frameC,"Alerting caregiver in {:.1f}s".format(NOTIFICATION_SECONDS - elapsed),(10,50), cv2.FONT_HERSHEY_SIMPLEX,0.75,(255,255,255),1,cv2.LINE_AA)                    
                else:
                    cv2.putText(frameC,"Resetting alert in {:.1f}s".format(NOTIFICATION_SECONDS - elapsed),(10,50), cv2.FONT_HERSHEY_SIMPLEX,0.75,(255,255,255),1,cv2.LINE_AA)                    
                #blink_count+=1
                if elapsed > NOTIFICATION_SECONDS:
                    if not block_notification:
                        telegram_bot_sendtext("Caregiver attention requested!".format(NOTIFICATION_SECONDS))
                        block_notification = True                
                    else:
                        block_notification = False
                    


                #telegram_bot_sendtext("Blinking detected")
        else:
            blink_count = 0
            is_blinking = False
            if block_notification:
                  cv2.putText(frameC,"* Caregiver Alert Dispatched *",(20,50), cv2.FONT_HERSHEY_SIMPLEX,0.75,(255,255,255),1,cv2.LINE_AA)    

      

    cv2.imshow('BlinkDetector', frameC)
    key = cv2.waitKey(1)
    if key == 27:
        break

#releasing the VideoCapture object
cap.release()
cv2.destroyAllWindows()